In [1]:
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# JSON 파일 경로
json_path = "data/structured_inu_final.json"

# 🔥 JSONLoader에서 필요한 필드를 불러오기
loader = JSONLoader(
    file_path=json_path,
    jq_schema='[.departments[], .faculty[], .research[], .academic_info[], .scholarships[], .admissions[], .campus_life[], .events[], .notices[] | {title: (.title // .name // "Untitled"), content: (.details // .description // "No details available"), category: (.category // "Unknown")}]',
    text_content=False  # 🔥 리스트 반환 문제 방지
)

# 데이터 로드
docs = loader.load()

# 🔥 리스트 문제 해결: content 필드를 문자열로 변환
formatted_docs = []
for doc in docs:
    content_data = doc.page_content

    # 🔥 만약 content가 리스트라면, 문자열로 변환
    if isinstance(content_data, list):
        content_data = "\n".join(content_data)  # 리스트를 줄바꿈으로 연결하여 문자열로 변환

    formatted_docs.append({
        "title": doc.metadata.get("title", "Untitled"),
        "content": content_data,
        "category": doc.metadata.get("category", "Unknown")
    })

# 🔥 데이터 확인 (처음 5개만 출력)
for i, doc in enumerate(formatted_docs[:5]):
    print(f"🔹 Document {i+1}:")
    print(f"Title: {doc['title']}")
    print(f"Content: {doc['content'][:500]}")  # 긴 내용은 앞부분만 출력
    print(f"Category: {doc['category']}")
    print("=" * 80)


🔹 Document 1:
Title: Untitled
Content: [{"title": "\uc815\ubcf4\uae30\uc220\ub300\ud559\ucef4\ud4e8\ud130\uacf5\ud559\ubd80", "content": "\ub85c\uadf8\uc778 , \ud559\ubd80\uc548\ub0b4 \ud559\ubd80\uc18c\uac1c \uad50\uc721\uc774\ub150\ubc0f\ubaa9\ud45c \uc5f0\ud601 \uad50\uc218\uc18c\uac1c \uc2e0\uc784\uad50\uc6d0\ucd08\ube59 \ud559\uacfc\uc0ac\ubb34\uc2e4\uc18c\uac1c \ucc3e\uc544\uc624\uc2dc\ub294\uae38 \ud559\uc0ac\ud589\uc815 \uad50\uc721\uacfc\uc815 \uad50\uacfc\ubaa9\uac1c\uc694 \uc878\uc5c5\uc694\uac74 \uc7a5\ud559\uae08 \ud559\
Category: Unknown


In [1]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# OpenAI의 "cl100k_base" 토크나이저 사용 (GPT-4, GPT-3.5에 적합)
tokenizer = tiktoken.get_encoding("cl100k_base")

# 텍스트 분할기 설정 (chunk 크기와 오버랩 설정)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=80,
)

# 텍스트 파일들이 저장된 폴더 경로 지정 (예: "./data")
loader = DirectoryLoader(
    "./data",          # 문서들이 위치한 폴더
    glob="structured_inu_data_clean.txt",     # txt 파일만 선택
    loader_cls=TextLoader  # 각 파일을 읽을 때 사용할 로더 클래스 지정
)

# 폴더 내 모든 텍스트 문서를 로드하고 텍스트 분할 적용
document_list = loader.load_and_split(text_splitter=text_splitter)

# 전체 토큰 수 계산
total_tokens = sum(len(tokenizer.encode(chunk.page_content)) for chunk in document_list)

print(f"📌 총 토큰 수: {total_tokens}")


📌 총 토큰 수: 36409211


In [2]:
len(document_list)

58619

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (chunk 크기와 오버랩 설정)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# DOCX 파일들이 저장된 폴더 경로 지정 (예: "./docs")
loader = DirectoryLoader(
    "./data",          # 문서들이 위치한 폴더
    glob="*.docx",     # docx 파일만 선택
    loader_cls=Docx2txtLoader  # 각 파일을 읽을 때 사용할 로더 클래스 지정
)

# 폴더 내 모든 DOCX 문서를 로드하고 텍스트 분할 적용
document_list = loader.load_and_split(text_splitter=text_splitter)


In [5]:
len(document_list)

14304

In [8]:
import tiktoken
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# OpenAI의 "cl100k_base" 토크나이저 사용 (GPT-4, GPT-3.5에 적합)
tokenizer = tiktoken.get_encoding("cl100k_base")

# 텍스트 분할기 설정 (chunk 크기와 오버랩 설정)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 텍스트 파일들이 저장된 폴더 경로 지정 (예: "./data")
loader = DirectoryLoader(
    "./data",          # 문서들이 위치한 폴더
    glob="*.txt",     # txt 파일만 선택
    loader_cls=TextLoader  # 각 파일을 읽을 때 사용할 로더 클래스 지정
)

# 폴더 내 모든 텍스트 문서를 로드하고 텍스트 분할 적용
document_list = loader.load_and_split(text_splitter=text_splitter)

# 전체 토큰 수 계산
total_tokens = sum(len(tokenizer.encode(chunk.page_content)) for chunk in document_list)

print(f"📌 총 토큰 수: {total_tokens}")


📌 총 토큰 수: 13878536


In [4]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [10]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings

# # 환경변수를 불러옴
# load_dotenv()

# # OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
# embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [4]:
embedding

UpstageEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f1ba229cbb0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f1ba229f730>, model='solar-embedding-1-large', dimensions=None, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar', embedding_ctx_length=4096, embed_batch_size=10, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, http_client=None, http_async_client=None)

In [5]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-inu-36M', persist_directory="./chroma_inu-36M")

# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma_inu", embedding_function=embedding)

In [14]:
query = '졸업학점은 몇 점 인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
# retrieved_docs = database.similarity_search(query, k=3)
retriever = database.as_retriever(    search_kwargs={"k": 3},)

In [15]:
retriever.invoke(query)

[Document(id='f10d114c-cde4-4b4a-bde2-3127c33331c9', metadata={'source': './inu_ER.docx'}, page_content='② 학사과정 시 본교 대학원의 교과목을 최대 12학점까지 별도로 취득할 수 있으며 그 학점 및 성적은 대학원에서 관리한다.<개정 2023.9.1.>\n\n③ 제2항에 따라 취득한 학점은 최대 6학점의 범위에서 학사과정 전공심화 학점으로 중복하여 인정할 수 있다. 다만, 학과에서 석사과정 연계 전공과 유사하다고 판단한 경우에 한하고, 상이하다고 판단한 경우 또는 학생이 학·석사 연계과정을 포기한 경우에는 일반선택 학점으로 인정할 수 있다.<개정 2022.7.8., 2023.9.1.>\n\n④ 연계과정에 선발된 학생은 대학원 등록 전까지 대학원 지도교수의 승인을 받은 연구활동중간보고서를 대학원에 제출하여야 한다.<개정 2023.9.1., 2024.7.31.>\n\n\n\n제98조(졸업) ① 학사과정의 졸업요건을 충족하고, 취득학점의 평점평균이 3.3 이상인 학생은 7학기 말에 졸업한다. 다만, 7학기 중에 대학원 입학을 위한 등록을 하여야 한다. <개정 2023.9.1.>\n\n② 제98조제1항을 이행하지 않는 학생은 학사과정의 졸업요건을 충족하였음에도 불구하고 제94조에 따라 졸업할 수 없으며, 학부 8학기 등록금을 납부하여야 한다. 다만, 학사과정의 졸업사정 기준에 의한 조기졸업 요건을 충족하는 경우에는 예외로 한다.<개정 2023.9.1.>\n\n③ 7학기에 학사과정의 졸업요건을 충족하고, 취득학점의 평점평균이 3.3 이상인 학생이 7학기 중에 대학원에 입학하지 못한 경우, 학부 8학기 등록금을 납부하고 8학기 중에 대학원 입학을 위한 등록을 하여야 하고, 8학기 말에 졸업하여 학·석사연계과정을 이어갈 수 있다.<신설 2024.7.31.>\n\n\n\n제99조(중도포기) ① 본 과정을 이수 중인 학생이 중도에 포기를 원하는 경우에는 마지막 학기말까지 포기신청서를 대학원 지도

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [29]:
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [30]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [31]:
ai_message = retrieval_chain.invoke({"input": query})

In [35]:
ai_message['answer']

'제공된 정보에서는 졸업학점이 130점부터 140점까지 다양하게 설정되어 있습니다. 정확한 졸업학점은 구체적인 학과나 프로그램에 따라 다를 수 있습니다.'